bag of words


In [4]:
from collections import defaultdict
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE
import gzip
import json
import string
import math

from sklearn.metrics import mean_squared_error
import random
from sklearn.model_selection import train_test_split

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boyiq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# from google.colab import drive
# drive.mount('/content/drive/')
# file_path = '/content/drive/My Drive/Digital_Music_5.json.gz'
file_path = 'Digital_Music_5.json.gz'

In [6]:
data = []
with gzip.open(file_path, 'rb') as file:
    for byte_line in file:
        line = byte_line.decode('utf-8').strip()
        review = json.loads(line)
        data.append(review)

In [7]:
data[0]

{'overall': 5.0,
 'vote': '3',
 'verified': True,
 'reviewTime': '06 3, 2013',
 'reviewerID': 'A2TYZ821XXK2YZ',
 'asin': '3426958910',
 'style': {'Format:': ' Audio CD'},
 'reviewerName': 'Garrett',
 'reviewText': 'This is awesome to listen to, A must-have for all Slayer fans..sadly needed to be a triple disc set..They have so many hits!!',
 'summary': 'Slayer Rules!',
 'unixReviewTime': 1370217600}

In [8]:
import pandas as pd
dataset = []
for d in data:
    if 'reviewText' not in d:
        continue
    if 'overall' not in d:
        d['overall'] = 5.0
    dataset.append({'reviewText':d['reviewText'], 'overall': d['overall']})
df = pd.DataFrame(dataset)

In [9]:
# preprocess the dataset's reviewtext by removing punctuation, stopwords


In [10]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
import re

def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boyiq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df.reviewText = df.reviewText.apply(lambda x : preprocess(x))

In [27]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import text

# Bag of Words with bigrams and limited features
text_transformer = CountVectorizer(ngram_range=(1, 2),
                                   max_features=1000)

# TF-IDF Transformation
tfidf_transformer = TfidfTransformer()

# Pipeline: CountVectorizer -> TfidfTransformer
pipeline = Pipeline([
    ('vect', text_transformer),
    ('tfidf', tfidf_transformer)
])


In [28]:
# apply the BOW vectorization and TF-IDF transformation
X_text = pipeline.fit_transform(df.reviewText)
# 9:1 train-test split
X_train, X_test, y_train, y_test = train_test_split(X_text, df.overall, test_size=0.1, random_state=42)

# K-Nearest Neighbors with Cosine Similarity

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
# Function to predict using cosine similarity with KNN
def predict_with_cosine_similarity(X_train, y_train, X_test, k=10):
    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(X_test, X_train)

    # Predict ratings
    predictions = []
    for similarity in similarity_matrix:
        # Get top k indices of most similar items
        top_k_indices = np.argsort(similarity)[-k:]
        # Compute the mean of the k nearest neighbors
        predictions.append(np.mean(y_train.iloc[top_k_indices]))

    return predictions


In [ ]:

# Predict ratings for test data
y_pred_cosine = predict_with_cosine_similarity(X_train, y_train, X_test, k=10)

In [ ]:
# write the predicted ratings to a file
with open('cosine_predictions.txt', 'w') as f:
    for p in y_pred_cosine:
        f.write(str(p) + '\n')
        

In [ ]:

# calculate mse score pairwise
from sklearn.metrics import mean_squared_error
mse_knn = mean_squared_error(y_test, y_pred_cosine)
print(mse_knn)

0.39787773389141073


# SVM on tfidf

In [ ]:
from sklearn.svm import SVC

# Create and train the SVM model
svm_model = SVC(kernel='linear') # try other kenrnels
svm_model.fit(X_train, y_train)

# Predict ratings for test data
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
mse_svm = mean_squared_error(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {mse_svm}")

0.49394870642030075


# Multinomial Naive Bayes Classifier on TF-IDF

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error

alpha = [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0,100.0,1000.0]

best_alpha = 0
best_score = -10000000
# 10-fold cross validation
for alp in alpha:
    nb_model = MultinomialNB(alpha=alp)
    scores = cross_val_score(nb_model, X_train, y_train, cv=cv_num, scoring='neg_mean_squared_error')
    score = np.mean(scores)
    if score > best_score:
        best_score = score
        best_alpha = alp
nb_model = MultinomialNB(alpha=best_alpha)
nb_model.fit(X_train, y_train)

# Predict ratings for test data
y_pred_nb = nb_model.predict(X_test)

# Evaluate the model (optional)
mse_nb = mean_squared_error(y_test, y_pred_nb)
print(f"Naive Bayes mse: {mse_nb}, best alpha: {best_alpha}")



Naive Bayes mse: 0.5506101515062194, best alpha: 0.001


# linear Regressor on Bow

In [32]:
from sklearn.linear_model import Ridge
alpha = [0.05, 0.1, 1,0,5.0, 10.0,100.0,1000.0]
cv_num = 2


best_alpha = 0
best_score = -100000
for alp in alpha:
    ridge_model = Ridge(alpha=alp)
    scores = cross_val_score(ridge_model, X_train, y_train, cv=cv_num, scoring='neg_mean_squared_error')
    score = np.mean(scores)
    # print("score", score)
    # print("alpha", alp)
    if score > best_score:
        best_score = score
        best_alpha = alp

ridge_model = Ridge(alpha=best_alpha)
ridge_model.fit(X_train, y_train)


from sklearn.metrics import mean_squared_error
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print(f"Ridge Regression mse: {mse_ridge}, alpha =  {best_alpha}")


Ridge Regression mse: 0.39684467921719774, alpha =  5.0


In [33]:
from sklearn.linear_model import Lasso

best_alpha = 0
best_score = -100000
for alp in alpha:
    lasso_model = Lasso(alpha=alp)
    scores = cross_val_score(lasso_model, X_train, y_train, cv=cv_num, scoring='neg_mean_squared_error')
    score = np.mean(scores)
    if score > best_score:
        best_score = score
        best_alpha = alp

lasso_model = Lasso(alpha=best_alpha)
lasso_model.fit(X_train, y_train)


y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f"Lasso Regression mse: {mse_lasso}, alpha =  {best_alpha}")




C:\Users\boyiq\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\boyiq\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15531.787545617954, tolerance: 3.9488917542250808
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\boyiq\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well

In [ ]:
from sklearn.linear_model import ElasticNet

set_of_alphas = [0.1, 0.5, 1.0, 10.0,100.0, 1000.0]
set_of_l1_ratios = [0.1, 0.5, 0.9]
best_score = -10000
best_alpha = 0
best_l1_ratio = 0
for alpha in set_of_alphas:
    for l1_ratio in set_of_l1_ratios:
        elastic_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        scores = cross_val_score(elastic_model, X_train, y_train, cv=cv_num, scoring='neg_mean_squared_error')
        if scores.mean() > best_score:
            best_score = scores.mean()
            best_alpha = alpha
            best_l1_ratio = l1_ratio
elastic_model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
elastic_model.fit(X_train, y_train)

y_pred_elastic = elastic_model.predict(X_test)
mse_elastic = mean_squared_error(y_test, y_pred_elastic)

print(f"Elastic Net Regression mse: {mse_elastic}")
print(f"Best alpha: {best_alpha}")
print(f"Best l1_ratio: {best_l1_ratio}")

Elastic Net Regression mse: 0.49258261376896556
Best alpha: 100.0
Best l1_ratio: 0.1
